In [7]:
# install modules: python-pptx, reportlab

from pptx import Presentation
from pptx.dml.color import RGBColor
from pptx.enum.dml import MSO_THEME_COLOR
from pptx.enum.text import MSO_ANCHOR, MSO_AUTO_SIZE
from pptx.util import Inches, Pt
from pptx.enum.text import PP_ALIGN
from pptx.enum.shapes import MSO_SHAPE

from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.ttfonts import TTFont
from reportlab import rl_config
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
from io import BytesIO

import os

In [ ]:
# Felépítés:
# 1. Cím diát hagyd üresen
# 2. Igék/énekek
# 3. Hírdetések átmásolása

In [ ]:
import copy

DIR_PATH = os.path.dirname(os.path.realpath(__file__))

# Modify it so that slideIndex is a list of slides to copy
def SlideCopyFromPasteInto(copyFromPres, slideIndex,  pasteIntoPres):

    # specify the slide you want to copy the contents from
    slide_to_copy = copyFromPres.slides[slideIndex]

    # Define the layout you want to use from your generated pptx

    slide_layout = pasteIntoPres.slide_layouts.get_by_name("Blank") # names of layouts can be found here under step 3: https://www.geeksforgeeks.org/how-to-change-slide-layout-in-ms-powerpoint/
    # it is important for slide_layout to be blank since you dont want these "Write your title here" or something like that textboxes
    # alternative: slide_layout = pasteIntoPres.slide_layouts[copyFromPres.slide_layouts.index(slide_to_copy.slide_layout)]
    
    # create now slide, to copy contents to 
    new_slide = pasteIntoPres.slides.add_slide(slide_layout)

    # create images dict
    imgDict = {}

    # now copy contents from external slide, but do not copy slide properties
    # e.g. slide layouts, etc., because these would produce errors, as diplicate
    # entries might be generated
    for shp in slide_to_copy.shapes:
        if 'Picture' in shp.name:
            # save image
            with open(shp.name+'.jpg', 'wb') as f:
                f.write(shp.image.blob)

            # add image to dict
            imgDict[shp.name+'.jpg'] = [shp.left, shp.top, shp.width, shp.height]
        else:
            # create copy of elem
            el = shp.element
            newel = copy.deepcopy(el)

            # add elem to shape tree
            new_slide.shapes._spTree.insert_element_before(newel, 'p:extLst')
    
    # things added first will be covered by things added last => since I want pictures to be in foreground, I will add them after others elements
    # you can change this if you want
    # add pictures
    for k, v in imgDict.items():
        new_slide.shapes.add_picture(k, v[0], v[1], v[2], v[3])
        os.remove(k)

    return new_slide # this returns slide so you can instantly work with it when it is pasted in presentation

In [32]:
def get_text_width(text, font_size, font_name='Calibri'):
    # Create a PDF document
    pdf_buffer = BytesIO()
    pdf_canvas = canvas.Canvas(pdf_buffer, pagesize=letter)
    pdf_canvas.setFont(font_name, font_size)

    # Draw the text on the PDF
    pdf_text_width = pdf_canvas.stringWidth(text, font_name, font_size)

    # Close the PDF
    pdf_canvas.save()

    return Inches(pdf_text_width / 72.0)  # Convert from points to inches (1 inch = 72 points)

def get_next_stop(vers_text,font_size,font_type,max_width):
    words_len = 0
    words_list = vers_text.split(" ")
    curr_words = words_list[0]
    stop_ind = len(curr_words)
    
    for word in words_list[1:]:
        curr_words += " " + word
        words_len = get_text_width(curr_words,font_size,font_type)
        #print(words_len,max_width,stop_ind,curr_words)
        #print(words_len/914400)
        
        if words_len > max_width:
            return stop_ind
        else:
            stop_ind = len(curr_words)
        
    return stop_ind

def get_scaling_factor(width):
    txt = 'a' * 37
    # 27 'a' can be fitted on linux, 37 on Windows
    return width/get_text_width(txt,36,'CalibriBd')

#TODO: there should be an inner function, called with vers_text so that it can call a new one if text is too long.
def create_bible_vers_slide(prs,vers_place):
    title_only_slide_layout = prs.slide_layouts[6]

    OFFSET = Inches(0.3)

    slide = prs.slides.add_slide(title_only_slide_layout)
    slide_w = prs.slide_width - OFFSET

    # Set black background
    background = slide.background
    fill = background.fill
    fill.solid()
    fill.fore_color.rgb = RGBColor(0, 0, 0)

    # Igehely
    left = OFFSET
    top = OFFSET
    height = Inches(1)
    width = slide_w - left

    SCALING_FACTOR = get_scaling_factor(width)

    vers_place_box = slide.shapes.add_shape(
        MSO_SHAPE.RECTANGLE, left, top, width, height
    )

    vers_place_box.fill.background()
    vers_place_box.line.fill.background()

    tf = vers_place_box.text_frame
    p = tf.paragraphs[0]
    run = p.add_run()
    run.text = vers_place
    font = run.font
    font.name = 'Calibri'
    font.size = Pt(28)
    font.bold = True
    #font.color.theme_color = MSO_THEME_COLOR.ACCENT_1
    font.color.rgb = RGBColor(255,255,255)
    p.alignment = PP_ALIGN.RIGHT

    # Igerész
    vers_font_size = 36
    left = OFFSET
    top = OFFSET + Pt(26) + OFFSET
    height = prs.slide_height - OFFSET - top
    width = slide_w - left

    vers_cont_box = slide.shapes.add_shape(
        MSO_SHAPE.RECTANGLE, left, top, width, height
    )

    # Hide box
    vers_cont_box.fill.background()
    vers_cont_box.line.fill.background()

    # Set up textbox
    tf = vers_cont_box.text_frame
    tf.word_wrap = True
    tf.vertical_anchor = MSO_ANCHOR.TOP
    p = tf.paragraphs[0]
    p.alignment = PP_ALIGN.LEFT
    run = p.add_run()

    # Enter text
    vers_text = 'Perferendies asdfa adfadf sdafdsasdf adsfdf id voluptatem maxime. Vero debitis dolorem iste blanditiis ut accusamus consectetur omnis. Maiores quasi et rerum voluptate aperiam ut nisi nihil. Quos laborum hic nihil. Nihil perferendis id quia. Minima incidunt molestiae laboriosam ut unde odit quos dolores.…'
    #vers_text = 'Perferendis id voluptatem maxime. Vero debitis dolorem iste blanditiis ut accusamus consectetur omnis. Maiores quasi et rerum voluptate aperiam ut nisi nihil. Quos laborum hic nihil. Nihil perferendis id quia. Minima incidunt molestiae laboriosam ut unde odit quos dolores asdfasaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa.…'
    #vers_text = 'a' *37
    # Only 4 lines are allowed
    max_lines = 4
    num_lines = 0

    vers_text_width = get_text_width(vers_text, vers_font_size, 'CalibriBd')

    print(width/914400)

    while(vers_text_width > width):
        stop_ind = get_next_stop(vers_text,vers_font_size,'CalibriBd', width / SCALING_FACTOR)
        #stop_ind = vers_text[:max_text_len].rfind(" ")
        run.text += vers_text[:stop_ind] + '\n'
        # Drop spaces
        vers_text = vers_text[stop_ind+1:]
        
        # Add text
        num_lines += 1
        
        # If we end with 10 lines, 1 more will be appended...
        if num_lines >= max_lines:
            # Create new slide starting from the current verse?
            # Or do you just continue?
            print("New slide required")
            #create_bible_vers_slide(prs,vers_place)
            break
            
        vers_text_width = get_text_width(vers_text, vers_font_size, 'CalibriBd')

    run.text += vers_text

    print(run.text)

    # Set font properties
    font = run.font
    font.name = 'Calibri'
    # 36
    font.size = Pt(vers_font_size)
    font.bold = True
    font.color.rgb = RGBColor(255,255,255)

In [12]:
# Create a class for a working textbox
# Create a function that creates slides for a given Bible vers

In [33]:
# Load Calibri font
rl_config.TTFSearchPath.append('./calibri-font-family')

pdfmetrics.registerFont(TTFont('Calibri', 'calibri-regular.ttf'))
pdfmetrics.registerFont(TTFont('CalibriBd', 'calibri-bold.ttf'))
pdfmetrics.registerFont(TTFont('CalibriIt', 'calibri-italic.ttf'))
pdfmetrics.registerFont(TTFont('CalibriBI', 'calibri-bold-italic.ttf'))

pdfmetrics.registerFontFamily('Calibri',normal='Calibri',bold='CalibriBd',italic='CalibriIt',boldItalic='CalibriBI')

rl_config.TTFSearchPath.remove('./calibri-font-family')

# This has to be guessed by many test runs
# Between 1.3 and 1.5
# 1.4 seems most fitting so far for linux
# Between 1.1 and 1.2 for windows
#SCALING_FACTOR = 1.4
#SCALING_FACTOR = 1.2

## Create the presentation

prs = Presentation()

create_bible_vers_slide(prs,"Zsolt 38")

# Save file
prs.save('test.pptx')

9.4
New slide required
Perferendies asdfa adfadf sdafdsasdf adsfdf
id voluptatem maxime. Vero debitis
dolorem iste blanditiis ut accusamus
consectetur omnis. Maiores quasi et rerum
voluptate aperiam ut nisi nihil. Quos laborum hic nihil. Nihil perferendis id quia. Minima incidunt molestiae laboriosam ut unde odit quos dolores.…


In [296]:
# Example usage
text = "Example Text"
font_size = 14  # Replace with your desired font size
font_name = 'Calibri' # Replace with your desired font name

width = get_text_width(text, font_size, font_name)
print(f"The precise width of the text is {width} inches.")

The precise width of the text is 1.0559624565972223 inches.
